In [80]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [81]:
waiting_time_df = pd.read_csv('data/waiting_times.csv')
attendance_df = pd.read_csv('data/attendance.csv')
entity_df = pd.read_csv('data/entity_schedule.csv')
link_attraction_park_df = pd.read_csv('data/link_attraction_park.csv', sep=';')
weather_df = pd.read_csv('data/weather_data.csv')
parades_df = pd.read_excel('data/parade_night_show.xlsx')


In [82]:
# Put the date in the same format as waiting time in quarter of 15min
weather_df['dt_iso'] = weather_df['dt_iso']
weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'].str.slice(0,13), format='%Y-%m-%d %H')
weather_df = weather_df[weather_df['timezone'] == 3600]
weather_df['dt_iso+1'] = weather_df['dt_iso'].apply(lambda row: row + pd.Timedelta(hours=1))
weather_df['QUARTER'] = weather_df.apply(lambda row: pd.date_range(row['dt_iso'], row['dt_iso+1'], freq='15min'), axis=1)
weather_df = weather_df.explode('QUARTER')
weather_df = weather_df.drop(columns=['dt_iso', 'dt_iso+1', 'timezone'])
weather_df = weather_df.rename(columns={'QUARTER': 'date'})
weather_df = weather_df.drop_duplicates(subset='date', keep='first')
weather_df


,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,date
0,915148800,Custom location,48.873492,2.295104,8.33,NaN,3.39,5.28,8.14,9.32,1006,NaN,NaN,71,5.57,152,NaN,NaN,NaN,NaN,NaN,8,800,Clear,sky is clear,01n,1999-01-01 00:00:00
0,915148800,Custom location,48.873492,2.295104,8.33,NaN,3.39,5.28,8.14,9.32,1006,NaN,NaN,71,5.57,152,NaN,NaN,NaN,NaN,NaN,8,800,Clear,sky is clear,01n,1999-01-01 00:15:00
0,915148800,Custom location,48.873492,2.295104,8.33,NaN,3.39,5.28,8.14,9.32,1006,NaN,NaN,71,5.57,152,NaN,NaN,NaN,NaN,NaN,8,800,Clear,sky is clear,01n,1999-01-01 00:30:00
0,915148800,Custom location,48.873492,2.295104,8.33,NaN,3.39,5.28,8.14,9.32,1006,NaN,NaN,71,5.57,152,NaN,NaN,NaN,NaN,NaN,8,800,Clear,sky is clear,01n,1999-01-01 00:45:00
0,915148800,Custom location,48.873492,2.295104,8.33,NaN,3.39,5.28,8.14,9.32,1006,NaN,NaN,71,5.57,152,NaN,NaN,NaN,NaN,NaN,8,800,Clear,sky is clear,01n,1999-01-01 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203663,1648335600,Custom location,48.873492,2.295104,10.41,NaN,2.27,8.99,8.74,12.88,1028,NaN,NaN,57,3.93,55,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n,2022-03-27 00:00:00
203664,1648339200,Custom location,48.873492,2.295104,9.50,NaN,2.36,7.49,7.74,12.31,1028,NaN,NaN,61,3.78,57,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n,2022-03-27 00:15:00
203664,1648339200,Custom location,48.873492,2.295104,9.50,NaN,2.36,7.49,7.74,12.31,1028,NaN,NaN,61,3.78,57,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n,2022-03-27 00:30:00
203664,1648339200,Custom location,48.873492,2.295104,9.50,NaN,2.36,7.49,7.74,12.31,1028,NaN,NaN,61,3.78,57,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n,2022-03-27 00:45:00


In [83]:
# Merge the waiting time and weather data
waiting_time_df['FIN_TIME'] = waiting_time_df['FIN_TIME'].str.slice(0,16)
waiting_time_df['FIN_TIME'] = pd.to_datetime(waiting_time_df['FIN_TIME'], format='%Y-%m-%d %H:%M')
waiting_time_df['DEB_TIME'] = waiting_time_df['DEB_TIME'].str.slice(0,16)
waiting_time_df['DEB_TIME'] = pd.to_datetime(waiting_time_df['DEB_TIME'], format='%Y-%m-%d %H:%M')
merge_1 = waiting_time_df.merge(weather_df, left_on='FIN_TIME', right_on='date' , how='left')
merge_1

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,date
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,1.514840e+09,Custom location,48.873492,2.295104,6.80,NaN,4.29,2.64,6.52,7.47,1010.0,NaN,NaN,84.0,7.58,272.0,NaN,NaN,NaN,NaN,NaN,39.0,802.0,Clouds,scattered clouds,03n,2018-01-01 21:15:00
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,1.514833e+09,Custom location,48.873492,2.295104,7.28,NaN,4.59,3.03,6.70,7.87,1006.0,NaN,NaN,83.0,8.38,279.0,NaN,NaN,NaN,NaN,NaN,23.0,801.0,Clouds,few clouds,02n,2018-01-01 19:45:00
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,1.514844e+09,Custom location,48.873492,2.295104,6.79,NaN,3.59,2.68,6.52,7.37,1011.0,NaN,NaN,80.0,7.41,270.0,NaN,NaN,NaN,NaN,NaN,51.0,803.0,Clouds,broken clouds,04n,2018-01-01 22:45:00
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,1.514808e+09,Custom location,48.873492,2.295104,7.77,NaN,6.71,3.28,7.52,8.32,998.0,NaN,NaN,93.0,9.89,199.0,NaN,0.92,NaN,NaN,NaN,99.0,500.0,Rain,light rain,10d,2018-01-01 13:00:00
4,2018-01-01,2018-01-01 17:00:00,17,2018-01-01 17:15:00,Skyway,5,15.0,92.0,211.500,198.25,15,15,0,16.0,1.514826e+09,Custom location,48.873492,2.295104,7.76,NaN,5.57,3.30,7.31,7.94,1001.0,NaN,NaN,86.0,9.77,285.0,NaN,0.25,NaN,NaN,NaN,100.0,500.0,Rain,light rain,10n,2018-01-01 17:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [84]:
# Merge attendance and link attraction park data
merge_1 = merge_1.merge(link_attraction_park_df, left_on='ENTITY_DESCRIPTION_SHORT', right_on='ATTRACTION', how='left')
merge_1

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,date,ATTRACTION,PARK
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,1.514840e+09,Custom location,48.873492,2.295104,6.80,NaN,4.29,2.64,6.52,7.47,1010.0,NaN,NaN,84.0,7.58,272.0,NaN,NaN,NaN,NaN,NaN,39.0,802.0,Clouds,scattered clouds,03n,2018-01-01 21:15:00,Roller Coaster,PortAventura World
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,1.514833e+09,Custom location,48.873492,2.295104,7.28,NaN,4.59,3.03,6.70,7.87,1006.0,NaN,NaN,83.0,8.38,279.0,NaN,NaN,NaN,NaN,NaN,23.0,801.0,Clouds,few clouds,02n,2018-01-01 19:45:00,Bumper Cars,PortAventura World
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,1.514844e+09,Custom location,48.873492,2.295104,6.79,NaN,3.59,2.68,6.52,7.37,1011.0,NaN,NaN,80.0,7.41,270.0,NaN,NaN,NaN,NaN,NaN,51.0,803.0,Clouds,broken clouds,04n,2018-01-01 22:45:00,Rapids Ride,PortAventura World
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,1.514808e+09,Custom location,48.873492,2.295104,7.77,NaN,6.71,3.28,7.52,8.32,998.0,NaN,NaN,93.0,9.89,199.0,NaN,0.92,NaN,NaN,NaN,99.0,500.0,Rain,light rain,10d,2018-01-01 13:00:00,Crazy Dance,PortAventura World
4,2018-01-01,2018-01-01 17:00:00,17,2018-01-01 17:15:00,Skyway,5,15.0,92.0,211.500,198.25,15,15,0,16.0,1.514826e+09,Custom location,48.873492,2.295104,7.76,NaN,5.57,3.30,7.31,7.94,1001.0,NaN,NaN,86.0,9.77,285.0,NaN,0.25,NaN,NaN,NaN,100.0,500.0,Rain,light rain,10n,2018-01-01 17:15:00,Skyway,Tivoli Gardens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Himalaya Ride,PortAventura World
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Crazy Dance,PortAventura World
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Crazy Dance,PortAventura World
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Giga Coaster,PortAventura World


In [85]:
# Merge merge_1 and attendance data
merge_1 = merge_1.merge(attendance_df, left_on=['PARK','WORK_DATE'] , right_on=['FACILITY_NAME', 'USAGE_DATE'], how='left')
merge_1

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,date,ATTRACTION,PARK,USAGE_DATE,FACILITY_NAME,attendance
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,1.514840e+09,Custom location,48.873492,2.295104,6.80,NaN,4.29,2.64,6.52,7.47,1010.0,NaN,NaN,84.0,7.58,272.0,NaN,NaN,NaN,NaN,NaN,39.0,802.0,Clouds,scattered clouds,03n,2018-01-01 21:15:00,Roller Coaster,PortAventura World,NaN,NaN,NaN
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,1.514833e+09,Custom location,48.873492,2.295104,7.28,NaN,4.59,3.03,6.70,7.87,1006.0,NaN,NaN,83.0,8.38,279.0,NaN,NaN,NaN,NaN,NaN,23.0,801.0,Clouds,few clouds,02n,2018-01-01 19:45:00,Bumper Cars,PortAventura World,NaN,NaN,NaN
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,1.514844e+09,Custom location,48.873492,2.295104,6.79,NaN,3.59,2.68,6.52,7.37,1011.0,NaN,NaN,80.0,7.41,270.0,NaN,NaN,NaN,NaN,NaN,51.0,803.0,Clouds,broken clouds,04n,2018-01-01 22:45:00,Rapids Ride,PortAventura World,NaN,NaN,NaN
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,1.514808e+09,Custom location,48.873492,2.295104,7.77,NaN,6.71,3.28,7.52,8.32,998.0,NaN,NaN,93.0,9.89,199.0,NaN,0.92,NaN,NaN,NaN,99.0,500.0,Rain,light rain,10d,2018-01-01 13:00:00,Crazy Dance,PortAventura World,NaN,NaN,NaN
4,2018-01-01,2018-01-01 17:00:00,17,2018-01-01 17:15:00,Skyway,5,15.0,92.0,211.500,198.25,15,15,0,16.0,1.514826e+09,Custom location,48.873492,2.295104,7.76,NaN,5.57,3.30,7.31,7.94,1001.0,NaN,NaN,86.0,9.77,285.0,NaN,0.25,NaN,NaN,NaN,100.0,500.0,Rain,light rain,10n,2018-01-01 17:15:00,Skyway,Tivoli Gardens,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Himalaya Ride,PortAventura World,NaN,NaN,NaN
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Crazy Dance,PortAventura World,NaN,NaN,NaN
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Crazy Dance,PortAventura World,NaN,NaN,NaN
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Giga Coaster,PortAventura World,NaN,NaN,NaN


In [86]:
parades_df.dtypes

Unnamed: 0             int64
WORK_DATE     datetime64[ns]
NIGHT_SHOW            object
PARADE_1              object
PARADE_2              object
dtype: object

In [87]:
import pandas as pd
# Add ending date for Night show and Parade 1 and 2

parades_df = parades_df.rename(columns={'NIGHT_SHOW': 'NIGHT_SHOW_STARTING_HOUR',
                                        'PARADE_1': 'PARADE_1_STARTING_HOUR',
                                        'PARADE_2': 'PARADE_2_STARTING_HOUR'})

# Supposons que parades_df est votre DataFrame et a déjà été défini
parades_df['NIGHT_SHOW_STARTING_HOUR'] = pd.to_datetime(parades_df['NIGHT_SHOW_STARTING_HOUR'], format='%H:%M:%S')
parades_df['PARADE_1_STARTING_HOUR'] = pd.to_datetime(parades_df['PARADE_1_STARTING_HOUR'], format='%H:%M:%S')
parades_df['PARADE_2_STARTING_HOUR'] = pd.to_datetime(parades_df['PARADE_2_STARTING_HOUR'], format='%H:%M:%S')

# Assurez-vous que la colonne WORK_DATE est au format datetime
parades_df['WORK_DATE'] = pd.to_datetime(parades_df['WORK_DATE'])

# Fonction pour combiner les dates et heures tout en gérant les valeurs potentiellement invalides
def combine_date_hour(row, time_col):
    # Extraire les composants de la date
    year, month, day = row['WORK_DATE'].year, row['WORK_DATE'].month, row['WORK_DATE'].day
    
    # Convertir la colonne d'heure en datetime si ce n'est pas déjà fait
    time_val = pd.to_datetime(row[time_col], errors='coerce', format='%H:%M:%S')
    
    # Vérifier si time_val est NaT (Not a Time), auquel cas retourner la valeur originale
    if pd.isna(time_val):
        return None  # ou retourner row[time_col] si vous voulez garder l'heure originale
    
    # Extraire les composants de l'heure
    hour, minute, second = time_val.hour, time_val.minute, time_val.second
    
    # Créer et retourner le nouveau Timestamp
    return pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute, second=second)

# Appliquer la fonction pour chaque colonne d'heure
for col in ['NIGHT_SHOW_STARTING_HOUR', 'PARADE_1_STARTING_HOUR', 'PARADE_2_STARTING_HOUR']:
    parades_df[f'{col}'] = parades_df.apply(combine_date_hour, time_col=col, axis=1)

def adjust_hour(time):
    # Extraire les heures et les minutes
    hour = time.hour
    minute = time.minute
    
    # Appliquer les conditions spécifiées pour ajuster l'heure
    if hour == 18 and minute == 40:
        return time.replace(minute=30)
    elif hour == 18 and minute == 10:
        return time.replace(minute=0)
    elif hour == 20 and minute == 20:
        return time.replace(minute=30)
    elif hour == 19 and minute == 10:
        return time.replace(minute=0)
    elif hour == 11 and minute == 50:
        return time.replace(hour=12) and time.replace(minute=0)
    elif hour == 12 and minute == 10:
        return time.replace(minute=0)
    elif hour == 10 and minute == 50:
        return time.replace(minute=0) and time.replace(hour=11)
    elif hour == 12 and minute == 25:
        return time.replace(minute=30)
    elif hour == 12 and minute == 55:
        return time.replace(minute=0) and time.replace(hour=13)
    elif hour == 13 and minute == 55:
        return time.replace(minute=0) and time.replace(hour=14)
    else:
        return time


# Appliquer la fonction pour ajuster les heures
parades_df['NIGHT_SHOW_STARTING_HOUR'] = parades_df['NIGHT_SHOW_STARTING_HOUR'].apply(adjust_hour)
parades_df['PARADE_1_STARTING_HOUR'] = parades_df['PARADE_1_STARTING_HOUR'].apply(adjust_hour)
parades_df['PARADE_2_STARTING_HOUR'] = parades_df['PARADE_2_STARTING_HOUR'].apply(adjust_hour)


parades_df['PARADE_1_ENDING_HOUR'] = parades_df['PARADE_1_STARTING_HOUR'] + pd.Timedelta(minutes=30)
parades_df['PARADE_2_ENDING_HOUR'] = parades_df['PARADE_2_STARTING_HOUR'] + pd.Timedelta(minutes=30)
parades_df['NIGHT_SHOW_ENDING_HOUR'] = parades_df['NIGHT_SHOW_STARTING_HOUR'] + pd.Timedelta(minutes=30)


In [88]:
parades_df

,Unnamed: 0,WORK_DATE,NIGHT_SHOW_STARTING_HOUR,PARADE_1_STARTING_HOUR,PARADE_2_STARTING_HOUR,PARADE_1_ENDING_HOUR,PARADE_2_ENDING_HOUR,NIGHT_SHOW_ENDING_HOUR
0,0,2018-10-01,2018-10-01 20:00:00,2018-10-01 17:30:00,2018-10-01 12:00:00,2018-10-01 18:00:00,2018-10-01 12:30:00,2018-10-01 20:30:00
1,1,2018-10-02,2018-10-02 20:00:00,2018-10-02 17:30:00,2018-10-02 12:00:00,2018-10-02 18:00:00,2018-10-02 12:30:00,2018-10-02 20:30:00
2,2,2018-10-03,2018-10-03 20:00:00,2018-10-03 17:30:00,2018-10-03 12:00:00,2018-10-03 18:00:00,2018-10-03 12:30:00,2018-10-03 20:30:00
3,3,2018-10-04,2018-10-04 20:00:00,2018-10-04 17:30:00,2018-10-04 12:00:00,2018-10-04 18:00:00,2018-10-04 12:30:00,2018-10-04 20:30:00
4,4,2018-10-05,2018-10-05 20:00:00,2018-10-05 17:30:00,2018-10-05 12:00:00,2018-10-05 18:00:00,2018-10-05 12:30:00,2018-10-05 20:30:00
...,...,...,...,...,...,...,...,...
666,666,2022-08-14,2022-08-14 23:00:00,2022-08-14 17:30:00,NaT,2022-08-14 18:00:00,NaT,2022-08-14 23:30:00
667,667,2022-08-15,2022-08-15 23:00:00,2022-08-15 17:30:00,NaT,2022-08-15 18:00:00,NaT,2022-08-15 23:30:00
668,668,2022-08-16,2022-08-16 23:00:00,2022-08-16 17:30:00,NaT,2022-08-16 18:00:00,NaT,2022-08-16 23:30:00
669,669,2022-08-17,2022-08-17 23:00:00,2022-08-17 17:30:00,NaT,2022-08-17 18:00:00,NaT,2022-08-17 23:30:00


In [89]:
parade_1 = parades_df[['PARADE_1_STARTING_HOUR', 'PARADE_1_ENDING_HOUR']]
parade_1['DATE'] = parade_1.apply(lambda row: pd.date_range(row['PARADE_1_STARTING_HOUR'], row['PARADE_1_ENDING_HOUR'], freq='15min'), axis=1)
parade_1 = parade_1.explode('DATE')
parade_1['IS PARADE 1'] = 1
parade_1 = parade_1.drop(columns=['PARADE_1_STARTING_HOUR', 'PARADE_1_ENDING_HOUR'])
parade_1

/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_89862/4077507876.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parade_1['DATE'] = parade_1.apply(lambda row: pd.date_range(row['PARADE_1_STARTING_HOUR'], row['PARADE_1_ENDING_HOUR'], freq='15min'), axis=1)


,DATE,IS PARADE 1
0,2018-10-01 17:30:00,1
0,2018-10-01 17:45:00,1
0,2018-10-01 18:00:00,1
1,2018-10-02 17:30:00,1
1,2018-10-02 17:45:00,1
...,...,...
669,2022-08-17 17:45:00,1
669,2022-08-17 18:00:00,1
670,2022-08-18 17:30:00,1
670,2022-08-18 17:45:00,1


In [90]:
#Take off NaT values from parade_2
parade_2 = parades_df[['PARADE_2_STARTING_HOUR', 'PARADE_2_ENDING_HOUR']]
parade_2 = parade_2.dropna()
parade_2['DATE'] = parade_2.apply(lambda row: pd.date_range(row['PARADE_2_STARTING_HOUR'], row['PARADE_2_ENDING_HOUR'], freq='15min'), axis=1)
parade_2 = parade_2.explode('DATE')
parade_2['IS PARADE 2'] = 1
parade_2 = parade_2.drop(columns=['PARADE_2_STARTING_HOUR', 'PARADE_2_ENDING_HOUR'])
parade_2

,DATE,IS PARADE 2
0,2018-10-01 12:00:00,1
0,2018-10-01 12:15:00,1
0,2018-10-01 12:30:00,1
1,2018-10-02 12:00:00,1
1,2018-10-02 12:15:00,1
...,...,...
461,2020-01-05 12:15:00,1
461,2020-01-05 12:30:00,1
462,2020-01-06 11:30:00,1
462,2020-01-06 11:45:00,1


In [91]:
parades_df_night = parades_df[['NIGHT_SHOW_STARTING_HOUR', 'NIGHT_SHOW_ENDING_HOUR']]
parades_df_night['DATE'] = parades_df_night.apply(lambda row: pd.date_range(row['NIGHT_SHOW_STARTING_HOUR'], row['NIGHT_SHOW_ENDING_HOUR'], freq='15min'), axis=1)
parades_df_night = parades_df_night.explode('DATE')
parades_df_night['IS NIGHT SHOW'] = 1
parades_df_night = parades_df_night.drop(columns=['NIGHT_SHOW_STARTING_HOUR', 'NIGHT_SHOW_ENDING_HOUR'])
parades_df_night

/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_89862/3402374210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parades_df_night['DATE'] = parades_df_night.apply(lambda row: pd.date_range(row['NIGHT_SHOW_STARTING_HOUR'], row['NIGHT_SHOW_ENDING_HOUR'], freq='15min'), axis=1)


,DATE,IS NIGHT SHOW
0,2018-10-01 20:00:00,1
0,2018-10-01 20:15:00,1
0,2018-10-01 20:30:00,1
1,2018-10-02 20:00:00,1
1,2018-10-02 20:15:00,1
...,...,...
669,2022-08-17 23:15:00,1
669,2022-08-17 23:30:00,1
670,2022-08-18 23:00:00,1
670,2022-08-18 23:15:00,1


In [92]:
merge_1.head()

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,date,ATTRACTION,PARK,USAGE_DATE,FACILITY_NAME,attendance
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,1.514840e+09,Custom location,48.873492,2.295104,6.80,NaN,4.29,2.64,6.52,7.47,1010.0,NaN,NaN,84.0,7.58,272.0,NaN,NaN,NaN,NaN,NaN,39.0,802.0,Clouds,scattered clouds,03n,2018-01-01 21:15:00,Roller Coaster,PortAventura World,NaN,NaN,NaN
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,1.514833e+09,Custom location,48.873492,2.295104,7.28,NaN,4.59,3.03,6.70,7.87,1006.0,NaN,NaN,83.0,8.38,279.0,NaN,NaN,NaN,NaN,NaN,23.0,801.0,Clouds,few clouds,02n,2018-01-01 19:45:00,Bumper Cars,PortAventura World,NaN,NaN,NaN
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,1.514844e+09,Custom location,48.873492,2.295104,6.79,NaN,3.59,2.68,6.52,7.37,1011.0,NaN,NaN,80.0,7.41,270.0,NaN,NaN,NaN,NaN,NaN,51.0,803.0,Clouds,broken clouds,04n,2018-01-01 22:45:00,Rapids Ride,PortAventura World,NaN,NaN,NaN
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,1.514808e+09,Custom location,48.873492,2.295104,7.77,NaN,6.71,3.28,7.52,8.32,998.0,NaN,NaN,93.0,9.89,199.0,NaN,0.92,NaN,NaN,NaN,99.0,500.0,Rain,light rain,10d,2018-01-01 13:00:00,Crazy Dance,PortAventura World,NaN,NaN,NaN
4,2018-01-01,2018-01-01 17:00:00,17,2018-01-01 17:15:00,Skyway,5,15.0,92.0,211.500,198.25,15,15,0,16.0,1.514826e+09,Custom location,48.873492,2.295104,7.76,NaN,5.57,3.30,7.31,7.94,1001.0,NaN,NaN,86.0,9.77,285.0,NaN,0.25,NaN,NaN,NaN,100.0,500.0,Rain,light rain,10n,2018-01-01 17:15:00,Skyway,Tivoli Gardens,NaN,NaN,NaN


In [93]:
#Merge parades_df and merge_1
merge_1 = merge_1.merge(parades_df_night, left_on='DEB_TIME' , right_on='DATE', how='left')
merge_1 = merge_1.merge(parade_1, left_on='DEB_TIME' , right_on='DATE', how='left')
merge_1 = merge_1.merge(parade_2, left_on='DEB_TIME' , right_on='DATE', how='left')


In [94]:
# Filter on PortAventura World, Fill 0 for the NaN values and drop unnecessary columns
merge_1 = merge_1[merge_1['PARK'] == 'PortAventura World']
merge_1['IS NIGHT SHOW'].fillna(0, inplace=True)
merge_1['IS PARADE 1'].fillna(0, inplace=True)
merge_1['IS PARADE 2'].fillna(0, inplace=True)
merge_1.drop(columns= ['DATE_x', 'DATE_y', 'DATE','FACILITY_NAME', 'date', 'weather_icon', 'weather_id', 'USAGE_DATE', 'city_name', 'dt', 'lat', 'lon', 'PARK', 'ENTITY_DESCRIPTION_SHORT'], inplace=True)

In [95]:
merge_1

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,temp,visibility,dew_point,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,ATTRACTION,attendance,IS NIGHT SHOW,IS PARADE 1,IS PARADE 2
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,6.80,NaN,4.29,2.64,6.52,7.47,1010.0,NaN,NaN,84.0,7.58,272.0,NaN,NaN,NaN,NaN,NaN,39.0,Clouds,scattered clouds,Roller Coaster,NaN,0.0,0.0,0.0
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,7.28,NaN,4.59,3.03,6.70,7.87,1006.0,NaN,NaN,83.0,8.38,279.0,NaN,NaN,NaN,NaN,NaN,23.0,Clouds,few clouds,Bumper Cars,NaN,0.0,0.0,0.0
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,6.79,NaN,3.59,2.68,6.52,7.37,1011.0,NaN,NaN,80.0,7.41,270.0,NaN,NaN,NaN,NaN,NaN,51.0,Clouds,broken clouds,Rapids Ride,NaN,0.0,0.0,0.0
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,7.77,NaN,6.71,3.28,7.52,8.32,998.0,NaN,NaN,93.0,9.89,199.0,NaN,0.92,NaN,NaN,NaN,99.0,Rain,light rain,Crazy Dance,NaN,0.0,0.0,0.0
5,2018-01-01,2018-01-01 18:15:00,18,2018-01-01 18:30:00,Free Fall,50,3.0,0.0,0.000,0.00,0,0,0,3.0,7.33,NaN,5.15,2.99,6.86,7.77,1004.0,NaN,NaN,86.0,8.76,284.0,NaN,0.24,NaN,NaN,NaN,67.0,Rain,light rain,Free Fall,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Himalaya Ride,NaN,0.0,0.0,0.0
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crazy Dance,NaN,0.0,0.0,0.0
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crazy Dance,NaN,0.0,0.0,0.0
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Giga Coaster,NaN,0.0,0.0,0.0


In [96]:
#Donne moi le pourcantge de valeurs manquantes
colonne_nulle = merge_1.isnull().mean() * 100
colonne_nulle[colonne_nulle > 0]


temp                    59.144172
visibility             100.000000
dew_point               59.144172
feels_like              59.144172
temp_min                59.144172
temp_max                59.144172
pressure                59.144172
sea_level              100.000000
grnd_level             100.000000
humidity                59.144172
wind_speed              59.144172
wind_deg                59.144172
wind_gust              100.000000
rain_1h                 92.771817
rain_3h                100.000000
snow_1h                 99.451435
snow_3h                100.000000
clouds_all              59.144172
weather_main            59.144172
weather_description     59.144172
attendance              30.109574
dtype: float64

In [97]:
# Drop toutes les colonnes qui ont plus de 80% de valeurs manquantes
merge_1 = merge_1.drop(columns=colonne_nulle[colonne_nulle > 80].index)

In [98]:
merge_1

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,ATTRACTION,attendance,IS NIGHT SHOW,IS PARADE 1,IS PARADE 2
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,6.80,4.29,2.64,6.52,7.47,1010.0,84.0,7.58,272.0,39.0,Clouds,scattered clouds,Roller Coaster,NaN,0.0,0.0,0.0
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,7.28,4.59,3.03,6.70,7.87,1006.0,83.0,8.38,279.0,23.0,Clouds,few clouds,Bumper Cars,NaN,0.0,0.0,0.0
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,6.79,3.59,2.68,6.52,7.37,1011.0,80.0,7.41,270.0,51.0,Clouds,broken clouds,Rapids Ride,NaN,0.0,0.0,0.0
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,7.77,6.71,3.28,7.52,8.32,998.0,93.0,9.89,199.0,99.0,Rain,light rain,Crazy Dance,NaN,0.0,0.0,0.0
5,2018-01-01,2018-01-01 18:15:00,18,2018-01-01 18:30:00,Free Fall,50,3.0,0.0,0.000,0.00,0,0,0,3.0,7.33,5.15,2.99,6.86,7.77,1004.0,86.0,8.76,284.0,67.0,Rain,light rain,Free Fall,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Himalaya Ride,NaN,0.0,0.0,0.0
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crazy Dance,NaN,0.0,0.0,0.0
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crazy Dance,NaN,0.0,0.0,0.0
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Giga Coaster,NaN,0.0,0.0,0.0


In [100]:
#Donne moi le pourcantge de valeurs manquantes
colonne_nulle = merge_1.isnull().mean() * 100
colonne_nulle[colonne_nulle > 0]


temp                   59.144172
dew_point              59.144172
feels_like             59.144172
temp_min               59.144172
temp_max               59.144172
pressure               59.144172
humidity               59.144172
wind_speed             59.144172
wind_deg               59.144172
clouds_all             59.144172
weather_main           59.144172
weather_description    59.144172
attendance             30.109574
dtype: float64

In [105]:
# Inputation par 0 pour les valeurs manquantes pour attendance
merge_1['attendance'].fillna(0, inplace=True)

#Imputation de la moyenne pour les valeurs manquantes pour les autres colonnes
merge_1['temp'].fillna(merge_1['temp'].mean(), inplace=True)
merge_1['feels_like'].fillna(merge_1['feels_like'].mean(), inplace=True)
merge_1['temp_min'].fillna(merge_1['temp_min'].mean(), inplace=True)
merge_1['temp_max'].fillna(merge_1['temp_max'].mean(), inplace=True)
merge_1['pressure'].fillna(merge_1['pressure'].mean(), inplace=True)
merge_1['humidity'].fillna(merge_1['humidity'].mean(), inplace=True)
merge_1['wind_speed'].fillna(merge_1['wind_speed'].mean(), inplace=True)
merge_1['wind_deg'].fillna(merge_1['wind_deg'].mean(), inplace=True)
merge_1['clouds_all'].fillna(merge_1['clouds_all'].mean(), inplace=True)
merge_1['weather_main'].fillna(merge_1['clouds_all'].mean(), inplace=True)






In [106]:
#Donne moi le pourcantge de valeurs manquantes
colonne_nulle = merge_1.isnull().mean() * 100
colonne_nulle[colonne_nulle > 0]


dew_point              59.144172
weather_description    59.144172
dtype: float64

In [107]:
# Drop dew_point, weather_description
merge_1 = merge_1.drop(columns=['dew_point', 'weather_description'])
merge_1

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,ATTRACTION,attendance,IS NIGHT SHOW,IS PARADE 1,IS PARADE 2
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0,6.800000,2.640000,6.520000,7.470000,1010.000000,84.000000,7.580000,272.00000,39.000000,Clouds,Roller Coaster,0.0,0.0,0.0,0.0
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0,7.280000,3.030000,6.700000,7.870000,1006.000000,83.000000,8.380000,279.00000,23.000000,Clouds,Bumper Cars,0.0,0.0,0.0,0.0
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0,6.790000,2.680000,6.520000,7.370000,1011.000000,80.000000,7.410000,270.00000,51.000000,Clouds,Rapids Ride,0.0,0.0,0.0,0.0
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0,7.770000,3.280000,7.520000,8.320000,998.000000,93.000000,9.890000,199.00000,99.000000,Rain,Crazy Dance,0.0,0.0,0.0,0.0
5,2018-01-01,2018-01-01 18:15:00,18,2018-01-01 18:30:00,Free Fall,50,3.0,0.0,0.000,0.00,0,0,0,3.0,7.330000,2.990000,6.860000,7.770000,1004.000000,86.000000,8.760000,284.00000,67.000000,Rain,Free Fall,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Himalaya Ride,0.0,0.0,0.0,0.0
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Crazy Dance,0.0,0.0,0.0,0.0
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Crazy Dance,0.0,0.0,0.0,0.0
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Giga Coaster,0.0,0.0,0.0,0.0


In [108]:
#Donne moi le pourcantge de valeurs manquantes
colonne_nulle = merge_1.isnull().mean() * 100
colonne_nulle[colonne_nulle > 0]


Series([], dtype: float64)

In [114]:
merge_1.rename(columns={'WORK_DATE': 'DATE',
                        'DEB_TIME': 'START OF 1 RIDE FOR ATTRACTION',
                        'FIN_TIME': 'END OF 1 RIDE FOR ATTRACTION', 
                        'DEB_TIME_HOUR': 'HOUR START OF 1 RIDE',
                        'ENTITY_DESCRIPTION_SHORT': 'ATTRACTION NAME', 
                        'WAIT_TIME_MAX': 'WAIT TIME', 
                        'NB_UNITS': 'NB UNITS', 
                        'GUEST_CARRIED': 'GUEST CARRIED', 
                        'OPEN TIME': 'ATTRACTION TIME OPEN FOR 15MIN', 
                        'UP_TIME': 'REAL TIME ATTRACTION OPEN FOR 15MIN', 
                        'DOWNTIME': 'REAL TIME ATTRACTION CLOSES FOR 15MIN', 
                        'NB_MAX_UNIT': 'NB MAX UNITS',
                        'temp': 'TEMPERATURE', 
                        'feels_like': 'TEMPERATURE FELT',
                        'temp_min': 'TEMPERATURE MINIMUM', 
                        'temp_max': 'TEMPERATURE MAXIMUM', 
                        'pressure': 'PRESSURE', 
                        'humidity': 'HUMIDITY', 
                        'wind_speed': 'WIND SPEED', 
                        'wind_deg': 'WIND DEG', 
                        'clouds_all': 'CLOUD PERCENTAGE',
                        'weather_main': 'WEATHER MAIN',
                        'attendance': 'ATTENDANCE',
                        'ADJUST_CAPACITY': 'ADJUST CAPACITY' 
                    }, inplace=True)

In [116]:
merge_1 = merge_1.sort_values(by='START OF 1 RIDE FOR ATTRACTION')
merge_1

,DATE,START OF 1 RIDE FOR ATTRACTION,HOUR START OF 1 RIDE,END OF 1 RIDE FOR ATTRACTION,ATTRACTION NAME,WAIT TIME,NB UNITS,GUEST CARRIED,CAPACITY,ADJUST CAPACITY,OPEN_TIME,REAL TIME ATTRACTION OPEN FOR 15MIN,REAL TIME ATTRACTION CLOSES FOR 15MIN,NB MAX UNITS,TEMPERATURE,TEMPERATURE FELT,TEMPERATURE MINIMUM,TEMPERATURE MAXIMUM,PRESSURE,HUMIDITY,WIND SPEED,WIND DEG,CLOUD PERCENTAGE,WEATHER MAIN,ATTRACTION,ATTENDANCE,IS NIGHT SHOW,IS PARADE 1,IS PARADE 2
471,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Rapids Ride,0,2.0,0.0,0.000,0.00,0,0,0,2.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,Clouds,Rapids Ride,0.0,0.0,0.0,0.0
1420,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Merry Go Round,0,65.0,0.0,0.000,0.00,0,0,0,130.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,Clouds,Merry Go Round,0.0,0.0,0.0,0.0
680,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Bumper Cars,5,18.0,108.0,254.749,254.75,15,15,0,18.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,Clouds,Bumper Cars,0.0,0.0,0.0,0.0
778,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Go-Karts,0,2.0,0.0,0.000,0.00,0,0,0,4.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,Clouds,Go-Karts,0.0,0.0,0.0,0.0
1658,2018-01-01,2018-01-01 09:00:00,9,2018-01-01 09:15:00,Spinning Coaster,0,1.4,0.0,315.750,52.65,9,9,0,6.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,Clouds,Spinning Coaster,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507853,2022-08-18,2022-08-18 22:45:00,22,2022-08-18 23:00:00,Free Fall,0,1.0,0.0,0.000,0.00,0,0,0,3.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Free Fall,0.0,0.0,0.0,0.0
3508552,2022-08-18,2022-08-18 22:45:00,22,2022-08-18 23:00:00,Giant Wheel,10,5.0,252.0,503.750,503.75,15,15,0,5.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Giant Wheel,0.0,0.0,0.0,0.0
3509264,2022-08-18,2022-08-18 22:45:00,22,2022-08-18 23:00:00,Rapids Ride,0,2.0,0.0,0.000,0.00,0,0,0,2.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Rapids Ride,0.0,0.0,0.0,0.0
3508496,2022-08-18,2022-08-18 22:45:00,22,2022-08-18 23:00:00,Haunted House,0,9.0,0.0,0.000,0.00,0,0,0,9.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,70.637711,Haunted House,0.0,0.0,0.0,0.0


In [117]:
# Save the final dataframe
merge_1.to_csv('data/final_data_15min.csv', index=False)

In [118]:
grouped = merge_1.copy()

In [119]:
grouped.drop(columns=['START OF 1 RIDE FOR ATTRACTION', 'HOUR START OF 1 RIDE', 'END OF 1 RIDE FOR ATTRACTION', 'IS NIGHT SHOW', 'IS PARADE 1', 'IS PARADE 2', 'ATTRACTION', 'WEATHER MAIN'], inplace=True)

In [125]:
grouped.drop(columns=['ATTRACTION'], inplace=True)

In [126]:
grouped

,DATE,ATTRACTION NAME,WAIT TIME,NB UNITS,GUEST CARRIED,CAPACITY,ADJUST CAPACITY,OPEN_TIME,REAL TIME ATTRACTION OPEN FOR 15MIN,REAL TIME ATTRACTION CLOSES FOR 15MIN,NB MAX UNITS,TEMPERATURE,TEMPERATURE FELT,TEMPERATURE MINIMUM,TEMPERATURE MAXIMUM,PRESSURE,HUMIDITY,WIND SPEED,WIND DEG,CLOUD PERCENTAGE,ATTENDANCE
471,2018-01-01,Rapids Ride,0,2.0,0.0,0.000,0.00,0,0,0,2.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,0.0
1420,2018-01-01,Merry Go Round,0,65.0,0.0,0.000,0.00,0,0,0,130.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,0.0
680,2018-01-01,Bumper Cars,5,18.0,108.0,254.749,254.75,15,15,0,18.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,0.0
778,2018-01-01,Go-Karts,0,2.0,0.0,0.000,0.00,0,0,0,4.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,0.0
1658,2018-01-01,Spinning Coaster,0,1.4,0.0,315.750,52.65,9,9,0,6.0,6.760000,2.620000,6.310000,6.940000,1007.000000,83.000000,7.480000,196.00000,100.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3507853,2022-08-18,Free Fall,0,1.0,0.0,0.000,0.00,0,0,0,3.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,0.0
3508552,2022-08-18,Giant Wheel,10,5.0,252.0,503.750,503.75,15,15,0,5.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,0.0
3509264,2022-08-18,Rapids Ride,0,2.0,0.0,0.000,0.00,0,0,0,2.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,0.0
3508496,2022-08-18,Haunted House,0,9.0,0.0,0.000,0.00,0,0,0,9.0,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.22542,70.637711,0.0


In [127]:
grouped = grouped.groupby(['DATE', 'ATTRACTION NAME']).mean()

In [128]:
grouped = grouped.sort_values(by='DATE')
grouped

WAIT TIME   NB UNITS  GUEST CARRIED    CAPACITY  \
DATE       ATTRACTION NAME                                                    
2018-01-01 Bumper Cars       4.732143  18.000000     128.500004  219.872652   
           Zipline          10.000000  11.125000      36.491071   92.209821   
           Water Ride       15.535714  10.010714     122.321425  205.834161   
           Swing Ride       34.196429  11.976191      94.705357  211.680357   
           Superman Ride    12.500000   2.177380      38.857138  128.549107   
...                               ...        ...            ...         ...   
2022-08-18 Bungee Jump      19.375000   4.611904     198.625000  295.794643   
           Bumper Cars       7.946429  18.000000     160.178571  231.700286   
           Water Ride       23.125000   9.000000      95.785714  226.123527   
           Haunted House    18.750000   8.885714     115.910714  204.107143   
           Zipline          42.321429  12.000000      62.339286  101.250000   

                            ADJUST CAPACITY  OPEN_TIME  \
DATE       ATTRACTION NAME                               
2018-01-01 Bumper Cars           219.873513  12.946429   
           Zipline                79.536607  13.660714   
           Water Ride            186.788696  12.500000   
           Swing Ride            203.123214  13.107143   
           Superman Ride         112.491661   7.910714   
...                                     ...        ...   
2022-08-18 Bungee Jump           271.940179  14.464286   
           Bumper Cars           231.701196  13.642857   
           Water Ride            187.896421  13.732143   
           Haunted House         201.250000  13.607143   
           Zipline               101.250000  15.000000   

                            REAL TIME ATTRACTION OPEN FOR 15MIN  \
DATE       ATTRACTION NAME                                        
2018-01-01 Bumper Cars                                12.946429   
           Zipline                                    12.732143   
           Water Ride                                 12.500000   
           Swing Ride                                 12.607143   
           Superman Ride                               7.910714   
...                                                         ...   
2022-08-18 Bungee Jump                                14.464286   
           Bumper Cars                                13.642857   
           Water Ride                                 11.410714   
           Haunted House                              13.607143   
           Zipline                                    15.000000   

                            REAL TIME ATTRACTION CLOSES FOR 15MIN  \
DATE       ATTRACTION NAME                                          
2018-01-01 Bumper Cars                                   0.000000   
           Zipline                                       0.928571   
           Water Ride                                    0.000000   
           Swing Ride                                    0.500000   
           Superman Ride                                 0.000000   
...                                                           ...   
2022-08-18 Bungee Jump                                   0.000000   
           Bumper Cars                                   0.000000   
           Water Ride                                    2.321429   
           Haunted House                                 0.000000   
           Zipline                                       0.000000   

                            NB MAX UNITS  TEMPERATURE  TEMPERATURE FELT  \
DATE       ATTRACTION NAME                                                
2018-01-01 Bumper Cars              18.0     7.496429          3.279286   
           Zipline                  12.0     7.496429          3.279286   
           Water Ride               11.0     7.496429          3.279286   
           Swing Ride               12.0     7.496429          3.279286   
           Superman Ride 

In [129]:
grouped.to_csv('data/final_data_day.csv', index=True)

: 